<img src="../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

# Setup AmsterdamUMCdb
## Requirements
- Access to the AmsterdamUMCdb csv files: request access from [Amsterdam Medical Data Science](https://www.amsterdammedicaldatascience.nl/).
- Operating system: any OS capable of running Python and PostgreSQL, including Windows, macOS and Linux.
- Internal memory: 8GB should suffice for basic analysis and running the Jupyter notebooks. However, the recommended memory specification to run both PostgreSQL and the Jupyter Notebooks on the same machine is 16-32 GB.
- Disk space: Downloading and extracting the database files will require 90 GB of hard disk space. In addition, creating the PostgreSQL database requires about 128 GB of hard disk space and and an additional 144 GB for creating the indices to improve query performance. 

## 1. Install a Python distribution
We **strongly recommend** installing Python using Anaconda, a popular distribution that includes many useful modules for data science out-of-the-box. Install the (latest) Python 3.7 version distribution from [Anaconda's](https://www.anaconda.com/distribution) distribution page.

## 2. Install PostgreSQL
PostgreSQL is an open source database management system (DBMS), available for most operating systems, including Windows, macOS and Linux. We recommend the installation of the most recent version of PostgreSQL (version 12) from the PostgreSQL [download](https://www.postgresql.org/download/) page. Please note your password for the `postgres` superuser, and if you did not chose `postgres` as the password, you need to modify these settings in the [`config.SAMPLE.ini`](../config.SAMPLE.ini) file in the root of the repository. Save the file as [`config.ini`](../config.ini).

## 3. Install psycopg2 module
To connect to your postgreSQL server from Python, the [psycopg2](https://pypi.org/project/psycopg2/) package needs to be installed from the Anaconda Prompt/Shell using conda:

> conda install -c anaconda psycopg2

## 4. Clone the AmsterdamUMCdb GitHub respository
Clone or download the [AmsterdamUMCdb](https://github.com/AmsterdamUMC/AmsterdamUMCdb) repository from GitHub. 
Follow the instructions on GitHub's online step-by-step guide, if needed: https://help.github.com/en/github/creating-cloning-and-archiving-repositories/cloning-a-repository. 

## 5. Download the database files
Download the AmsterdamUMCdb zip file from and extract the files from the zip file to the [`data`](../data) folder of the cloned AmsterdamUMCdb repository.

## 6. Create database tables
Start the Jupyter notebook server from the command line (using Command Prompt on Windows or Terminal on Mac/Linux) by running

> jupyter notebook

From the Jupyter file browser, open the `setup-amsterdamumc.ipynb` file from the `setup-amsterdamumc` folder in the cloned repository. This code in the notebook assumes there is a default postgres installation with a dabase named `postgres`, user `postgres` with password `postgres`. You should change these settings in the [`config.SAMPLE.ini`](../config.SAMPLE.ini) file in the root of the repository and save the file as [`config.ini`](../config.ini).
To create the tables in the database run this Jupyter notebook, either cell by cell (▶️ Run) or use the ⏩ button to  perform all steps sequentially. An `amsterdamumc` [schema](https://www.postgresql.org/docs/12/ddl-schemas.html) will be created, and all tables will be added to this schema.

## 7. Verify the database
After this notebook has been run completely, the postgres database should contain all tables with the same number of records we released. The output should state `Verification: PASSED`. You can verify it [here](#verify).
 
## 8. Create database table indices
It's highly recommended to create some useful indices to improve performance for common queries on identifiers like admissionid, itemid and measured times. 

## 9. Jupyter Notebooks
While the indices are being created, the postgreSQL should be available for querying using the notebooks in the [`tables`](../tables) folder (with lower performance). We use  plotly (version >4) for interactive plots in some notebooks. Plotly can be installed by using conda:

> conda install -c plotly plotly

# Python settings
## Imports

In [1]:
%matplotlib inline
import psycopg2
import pandas as pd
import numpy as np

import io
import os
from IPython.display import display, HTML, Markdown, clear_output

from tqdm.auto import tqdm

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

class ProgressFile:
    """ProgressFile: simple class to work as a file-like object that can be used by function to read the file
    while at the same time updating a progress bar.
    """
   
    def __init__(self, pbar, filename, mode="r"):
        self.file = open(filename, mode, encoding='windows-1252')
        self.progress = self.file.tell()
        self.pbar = pbar
        self.file.readline() #skip the first line: csv header

    def close(self):
        self.file.seek(0, os.SEEK_END)
        self.pbar.n = self.file.tell()
        self.pbar.refresh()
        self.file.close()
        
    def read(self, size):
        buf = self.file.read(size)
        self.pbar.update(size)

        return buf
        
    def readline(self, size):
        buf = self.file.readline(size)
        self.pbar.update(size)
        
        return buf
    
    
def copy_progress(csv, table):
    """copy the csv file to the table using tdqm progressbar.
    """
    #import the database using a tqdm progressbar and a ProgressFile
    pbar = tqdm(total=os.path.getsize(csv), desc='Importing '+os.path.splitext(os.path.basename(csv))[0],
            dynamic_ncols=True, unit_scale=1, unit='Bytes') #make a tdqm progress bar object

    pfile = ProgressFile(pbar, csv, 'r') #create a ProgressFile for showing progress
     
    cursor.copy_expert("""COPY {} FROM STDIN WITH (FORMAT CSV)""".format(table), pfile)
        
    #close the objects
    pfile.close()
    pbar.close()

    #show the first 10 records of this table
    df = pd.read_sql('SELECT * FROM ' + table + ' LIMIT 10',con)
    display(Markdown('## ' + os.path.splitext(os.path.basename(csv))[0] + ' (' + str(cursor.rowcount) + ' records copied):\n'))
    display(df)

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../config.ini'):
    config.read('../config.ini')
else:
    config.read('../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)
cursor = con.cursor()

## Create schema
Create an `amsterdamumc` schema to prevent collissions with (possible) other tables in the default `public` schema and change the schema path to our newly created 'amsterdamumc' schema to access the data without schema qualifications (e.g. `admissions` instead of `amsterdamumc.admissions`.

In [4]:
sql = """
CREATE SCHEMA IF NOT EXISTS amsterdamumcdb;
GRANT USAGE ON SCHEMA amsterdamumcdb TO public;
GRANT CREATE ON SCHEMA amsterdamumcdb TO public;
SET SCHEMA 'amsterdamumcdb';
"""
cursor.execute(sql)

# Create admissions table

In [5]:
table = 'admissions'
sql = """
DROP TABLE IF EXISTS admissions CASCADE;
CREATE TABLE admissions 
(
    patientid INTEGER,
    admissionid serial PRIMARY KEY,
    admissioncount INTEGER,
    location VARCHAR,
    urgency BIT,
    origin VARCHAR,
    admittedat BIGINT,
    admissionyeargroup VARCHAR,
    dischargedat BIGINT,
    lengthofstay SMALLINT,
    destination VARCHAR,
    gender VARCHAR,
    agegroup VARCHAR,
    dateofdeath BIGINT,
    weightgroup VARCHAR,
    weightsource VARCHAR,
    heightgroup VARCHAR,
    heightsource VARCHAR,
    specialty VARCHAR
);
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing admissions:   0%|          | 0.00/2.86M [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## admissions (23106 records copied):


,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty
0,0,0,1,IC,0,None,0,2003-2009,148800000,42,16,Vrouw,80+,None,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie
1,1,1,1,IC,0,None,0,2010-2016,96120000,26,15,Man,60-69,None,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,15,Man,60-69,None,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie
3,3,3,1,IC,0,None,0,2003-2009,84900000,23,14,Man,50-59,None,90-99,None,180-189,Gemeten,Cardiochirurgie
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,19,Man,70-79,None,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie
5,5,5,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,246420000,69,31,Man,50-59,None,60-69,Geschat,160-169,Gemeten,Longziekte
6,6,6,1,IC,1,Verpleegafdeling ander ziekenhuis,0,2010-2016,173460000,48,45,Vrouw,80+,None,70-79,Geschat,160-169,Geschat,Neurochirurgie
7,7,7,1,MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,148680000,42,41,Man,70-79,None,70-79,Gemeten,170-179,Gemeten,Nefrologie
8,8,8,1,IC,0,None,0,2003-2009,84720000,24,15,Man,50-59,None,100-109,Geschat,170-179,Anamnestisch,Cardiochirurgie
9,9,9,1,IC,0,None,0,2003-2009,242040000,67,15,Vrouw,70-79,None,70-79,None,159-,None,Cardiochirurgie


# Create drugitems table

In [6]:
table = 'drugitems'
sql = """
DROP TABLE IF EXISTS drugitems CASCADE;
CREATE TABLE drugitems 
(
    admissionid INTEGER,
    orderid BIGINT,
    ordercategoryid INTEGER,
    ordercategory VARCHAR,
    itemid INTEGER,
    item VARCHAR,
    isadditive BIT,
    isconditional BIT,
    rate FLOAT,
    rateunit VARCHAR,
    rateunitid INTEGER,
    ratetimeunitid INTEGER,
    doserateperkg BIT,
    dose FLOAT,
    doseunit VARCHAR,
    doserateunit VARCHAR,
    doseunitid INTEGER,    
    doserateunitid INTEGER,
    administered FLOAT,
    administeredunit VARCHAR,
    administeredunitid INTEGER,
    action VARCHAR,
    start BIGINT,
    stop BIGINT,
    duration BIGINT,
    solutionitemid INTEGER,
    solutionitem VARCHAR,
    solutionadministered FLOAT,
    solutionadministeredunit VARCHAR,
    fluidin FLOAT,
    iscontinuous BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing drugitems:   0%|          | 0.00/819M [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## drugitems (4907269 records copied):


,admissionid,orderid,ordercategoryid,ordercategory,itemid,item,isadditive,isconditional,rate,rateunit,rateunitid,ratetimeunitid,doserateperkg,dose,doseunit,doserateunit,doseunitid,doserateunitid,administered,administeredunit,administeredunitid,action,start,stop,duration,solutionitemid,solutionitem,solutionadministered,solutionadministeredunit,fluidin,iscontinuous
0,0,1994485,15,Injecties Antimicrobiele middelen,9152,Cefazoline (Kefzol),0,0,0.0,None,None,None,0,1.0,g,None,9,NaN,1.0,g,9,Nieuwe toediening,20100000,20160000,1,8987.0,"NaCl 0,9% Medicatie",0.0,ml,0.0,0
1,0,1995066,15,Injecties Antimicrobiele middelen,9152,Cefazoline (Kefzol),0,0,0.0,None,None,None,0,1.0,g,None,9,NaN,1.0,g,9,Nieuwe toediening,52500000,52560000,1,8987.0,"NaCl 0,9% Medicatie",0.0,ml,0.0,0
2,0,1995793,15,Injecties Antimicrobiele middelen,9152,Cefazoline (Kefzol),0,0,0.0,None,None,None,0,1.0,g,None,9,NaN,1.0,g,9,Nieuwe toediening,84900000,84960000,1,8987.0,"NaCl 0,9% Medicatie",0.0,ml,0.0,0
3,0,1994602,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,30540000,34140000,60,NaN,None,NaN,None,500.0,0
4,0,1994826,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,45780000,49380000,60,NaN,None,NaN,None,500.0,0
5,0,1995728,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,1000.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,88140000,89940000,30,NaN,None,NaN,None,500.0,0
6,0,1996545,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,124560000,128160000,60,NaN,None,NaN,None,500.0,0
7,0,1994886,23,Injecties CZS/Sedatie/Analgetica,7225,Morfine,0,0,NaN,None,None,None,0,5.0,mg,None,10,NaN,5.0,mg,10,Nieuwe toediening,47580000,47640000,1,NaN,None,NaN,None,0.0,0
8,0,1995016,23,Injecties CZS/Sedatie/Analgetica,7225,Morfine,0,0,NaN,None,None,None,0,5.0,mg,None,10,NaN,5.0,mg,10,Nieuwe toediening,51660000,51720000,1,NaN,None,NaN,None,0.0,0
9,0,1995143,23,Injecties CZS/Sedatie/Analgetica,7225,Morfine,0,0,NaN,None,None,None,0,2.5,mg,None,10,NaN,2.5,mg,10,Nieuwe toediening,58680000,58740000,1,NaN,None,NaN,None,0.0,0


# Create freetextitems table

In [7]:
table = 'freetextitems'
sql = """
DROP TABLE IF EXISTS freetextitems CASCADE;
CREATE TABLE freetextitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    value VARCHAR,
    comment VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing freetextitems:   0%|          | 0.00/52.3M [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## freetextitems (651248 records copied):


,admissionid,itemid,item,value,comment,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult
0,0,10890,Eiwit Kwal (urine),NEGATIEF,None,-5340000,-5340000,Systeem,62921677,None,1
1,0,10906,Hb/Ery's (urine),SPOOR,None,-5340000,-5340000,Systeem,62921710,None,1
2,0,11646,Afname (bloed),ART.,None,-978000000,-978000000,Systeem,316843,None,1
3,1,10858,Treponema pallidum (ELISA) (bloed),volgt,~ Gecodeerde Aanvr. opm. :Pre-op,-97800000,-97800000,Systeem,14837173,None,1
4,1,10885,Grootte (bloed),ANISO+,None,-79140000,-79140000,Systeem,219220,None,1
5,1,10890,Eiwit Kwal (urine),+,Opmerking verwijderd,-86280000,-86280000,Systeem,236206,None,1
6,1,10891,Gluc.Kwal. (urine),NEGATIEF,Opmerking verwijderd,-86280000,-86280000,Systeem,233096,None,1
7,1,10906,Hb/Ery's (urine),NEGATIEF,Opmerking verwijderd,-86280000,-86280000,Systeem,236706,None,1
8,1,10948,spijt (bloed),volgt,~ Gecodeerde Aanvr. opm. :Pre-op,-97800000,-97800000,Systeem,14838003,None,1
9,1,11646,Afname (bloed),ART.,None,-13500000,-13500000,Systeem,176703,None,1


# Create listitems table

In [8]:
table = 'listitems'
sql = """
DROP TABLE IF EXISTS listitems CASCADE;
CREATE TABLE listitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    valueid INT,
    value VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing listitems:   0%|          | 0.00/2.84G [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## listitems (30744065 records copied):


,admissionid,itemid,item,valueid,value,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult
0,0,6671,Hartritme,1,NSR,20520000,21480000,ICV_IC-Verpleegkundig,21480000,ICV_IC-Verpleegkundig,0
1,0,6671,Hartritme,1,NSR,22320000,29400000,ICV_IC-Verpleegkundig,29400000,ICV_IC-Verpleegkundig,0
2,0,6671,Hartritme,1,NSR,25920000,29460000,ICV_IC-Verpleegkundig,29460000,ICV_IC-Verpleegkundig,0
3,0,6671,Hartritme,1,NSR,29520000,29460000,ICV_IC-Verpleegkundig,29460000,ICV_IC-Verpleegkundig,0
4,0,6671,Hartritme,1,NSR,33120000,33300000,ICV_IC-Verpleegkundig,33300000,ICV_IC-Verpleegkundig,0
5,0,6671,Hartritme,1,NSR,36720000,36960000,ICV_IC-Verpleegkundig,36960000,ICV_IC-Verpleegkundig,0
6,0,6671,Hartritme,1,NSR,40320000,40440000,ICV_IC-Verpleegkundig,40440000,ICV_IC-Verpleegkundig,0
7,0,6671,Hartritme,1,NSR,43920000,45540000,ICV_IC-Verpleegkundig,45540000,ICV_IC-Verpleegkundig,0
8,0,6671,Hartritme,1,NSR,51120000,51300000,ICV_IC-Verpleegkundig,51300000,ICV_IC-Verpleegkundig,0
9,0,6671,Hartritme,1,NSR,58320000,62700000,ICV_IC-Verpleegkundig,62700000,ICV_IC-Verpleegkundig,0


# Create numericitems table
This is the largest table and can take a while depending on the performance of your system.

In [9]:
table = 'numericitems'
sql = """
    DROP TABLE IF EXISTS numericitems CASCADE;
    CREATE TABLE numericitems 
    (
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    tag VARCHAR,
    value FLOAT,
    unitid INT,
    unit VARCHAR,
    comment VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT,
    fluidout FLOAT
    )
"""
try:
    cursor.execute(sql)
except:
    print('There was an exception')
    

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing numericitems:   0%|          | 0.00/80.2G [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## numericitems (977625612 records copied):


,admissionid,itemid,item,tag,value,unitid,unit,comment,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult,fluidout
0,0,6640,Hartfrequentie,,83.0,15,/min,Opmerking verwijderd,20520000,20580000,ICV_IC-Verpleegkundig,20580000,ICV_IC-Verpleegkundig,0,None
1,0,6640,Hartfrequentie,,84.0,15,/min,Opmerking verwijderd,22320000,23700000,ICV_IC-Verpleegkundig,23700000,ICV_IC-Verpleegkundig,0,None
2,0,6640,Hartfrequentie,,82.0,15,/min,Opmerking verwijderd,25920000,26160000,ICV_IC-Verpleegkundig,26160000,ICV_IC-Verpleegkundig,0,None
3,0,6640,Hartfrequentie,,78.0,15,/min,Opmerking verwijderd,29520000,29460000,ICV_IC-Verpleegkundig,29469867,ICV_IC-Verpleegkundig,0,None
4,0,6640,Hartfrequentie,,73.0,15,/min,Opmerking verwijderd,33120000,33300000,ICV_IC-Verpleegkundig,33300000,ICV_IC-Verpleegkundig,0,None
5,0,6640,Hartfrequentie,,77.0,15,/min,Opmerking verwijderd,36720000,36960000,ICV_IC-Verpleegkundig,36960000,ICV_IC-Verpleegkundig,0,None
6,0,6640,Hartfrequentie,,78.0,15,/min,Opmerking verwijderd,40320000,40440000,ICV_IC-Verpleegkundig,40440000,ICV_IC-Verpleegkundig,0,None
7,0,6640,Hartfrequentie,,79.0,15,/min,Opmerking verwijderd,43920000,45540000,ICV_IC-Verpleegkundig,45540000,ICV_IC-Verpleegkundig,0,None
8,0,6640,Hartfrequentie,,88.0,15,/min,Opmerking verwijderd,47520000,49800000,ICV_IC-Verpleegkundig,49800000,ICV_IC-Verpleegkundig,0,None
9,0,6640,Hartfrequentie,,93.0,15,/min,Opmerking verwijderd,51120000,51300000,ICV_IC-Verpleegkundig,51300000,ICV_IC-Verpleegkundig,0,None


# Create procedureorderitems table

In [10]:
table = 'procedureorderitems'
sql = """
DROP TABLE IF EXISTS procedureorderitems CASCADE;
CREATE TABLE procedureorderitems 
(
    admissionid INTEGER,
    orderid BIGINT,
    ordercategoryid INT,
    ordercategoryname VARCHAR,
    itemid INT,
    item VARCHAR,
    registeredat BIGINT,
    registeredby VARCHAR
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing procedureorderitems:   0%|          | 0.00/208M [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## procedureorderitems (2188626 records copied):


,admissionid,orderid,ordercategoryid,ordercategoryname,itemid,item,registeredat,registeredby
0,0,1994567,73,Opdr. Laboratorium afname,9267,Lab. CCH-VAT T = 2,27840000,ICV_IC-Verpleegkundig
1,0,1994708,73,Opdr. Laboratorium afname,9268,Lab. CCH-VAT T = 6,40500000,ICV_IC-Verpleegkundig
2,0,1994480,73,Opdr. Laboratorium afname,9269,Lab. CCH-VAT T = 0,22680000,ICV_IC-Verpleegkundig
3,0,1994479,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,22680000,ICV_IC-Verpleegkundig
4,0,1994566,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,27840000,ICV_IC-Verpleegkundig
5,0,1994707,73,Opdr. Laboratorium afname,9270,Bloedgas afnemen,40500000,ICV_IC-Verpleegkundig
6,0,1994839,73,Opdr. Laboratorium afname,9446,Lab. dgs Standaard 6.00u,46020000,ICV_IC-Verpleegkundig
7,0,1995807,73,Opdr. Laboratorium afname,10752,Bloed afnemen,92580000,ICV_IC-Verpleegkundig
8,0,1994482,81,Opdr. Circulatie,7921,ECG,22680000,ICV_IC-Verpleegkundig
9,0,1994838,81,Opdr. Circulatie,9445,ECG 6.00u,46020000,ICV_IC-Verpleegkundig


# Create processitems table

In [11]:
table = 'processitems'
sql = """
DROP TABLE IF EXISTS processitems CASCADE;
CREATE TABLE processitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    start BIGINT,
    stop BIGINT,
    duration BIGINT
);
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

Importing processitems:   0%|          | 0.00/13.0M [00:00<?, ?Bytes/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## processitems (256715 records copied):


,admissionid,itemid,item,start,stop,duration
0,0,9159,Arterielijn Radialis,20520000,148800000,2138
1,0,9166,Trilumen Jugularis,20520000,148800000,2138
2,0,9174,Swan Ganz Jugularis,20520000,139020000,1975
3,0,9328,Beademen,20520000,69120000,810
4,0,9399,Wonddrain 1,20520000,94320000,1230
5,0,9400,Wonddrain 2,20520000,142080000,2026
6,0,9403,Thoraxdrain 1,20520000,148800000,2138
7,0,9415,Urine-Catheter (CAD),20520000,148800000,2138
8,0,9417,Sonde,20520000,148800000,2138
9,0,9422,Perifeer infuus,20520000,148800000,2138


# <a id='verify'></a>Verify record counts with published data
Compares the counts of the imported tables with our published number of records to verify that they match. Since importing in postgreSQL is already very strict using the COPY FROM command (same number of rows, compatible datatypes), an equal number of rows assumes a correct import.

In [12]:
#AmsterdamUMCdb Version 1.0.2 record counts
data = [
    ['admissions', 23106],
    ['drugitems', 4907269],
    ['freetextitems', 651248],
    ['listitems', 30744065],
    ['numericitems', 977625612],
    ['procedureorderitems', 2188626],
    ['processitems', 256715]
]

counts_published = pd.DataFrame(data, columns=['tables', 'counts'])
counts_published = counts_published.set_index('tables')

failed = False
html = u'<table style="font-size:16px" ><th style="text-align:left">Table<th>Counts postgres<th>Counts published<th style="text-align:center">Verified'
for table in counts_published.index:
        sql = "SELECT COUNT(admissionid) FROM " + table + ";"
        try:
            cursor.execute(sql)
            count = cursor.fetchone()[0]
        except:
            count = 0
        
        count_published = counts_published.loc[table, 'counts']
        
        if count == count_published:
            count_html = str(count)
            image = '<svg viewBox="0 0 12 16" version="1.1" width="24" height="32" aria-hidden="true">\
            <path fill="#28a745" fill-rule="evenodd" d="M12 5l-8 8-4-4 1.5-1.5L4 10l6.5-6.5L12 5z"></path></svg>'
        else:
            failed = True
            count_html = '<font color="#cb2431"><b>' + str(count) + '</b></font>'
            image = '<svg viewBox="0 0 12 16" version="1.1" width="24" height="32" aria-hidden="true">\
            <path fill="#cb2431" fill-rule="evenodd" d="M7.48 8l3.75 3.75-1.48 1.48L6 9.48l-3.75 3.75-1.48-1.48L4.52 \
            8 .77 4.25l1.48-1.48L6 6.52l3.75-3.75 1.48 1.48L7.48 8z"></path></svg>'
        
        html = html + '<tr><td style="text-align:left" width="256">' + table + '</td><td width="256">' + count_html + '</td><td width="256">' + str(count_published) + \
            '</td><td width="256" style="text-align:center">' + image + '</td></tr>'
        clear_output(wait=True)
        get_ipython().run_cell_magic(u'HTML', u'', html)

if failed:
    conclusion = '<tr></tr><tr bgcolor="#cb2431"><td style="text-align:left"><font style="font-size:30px" color="#ffffff"><b>Verification:</b></font></td><td></td><td></td><td><font style="font-size:30px" color="#ffffff"><b>FAILED!</b></font></td>'
else:
    conclusion = '<tr></tr><tr bgcolor="#28a745"><td style="text-align:left"><font style="font-size:30px" color="#ffffff"><b>Verification:</b></td></font><td></td><td></td><td><font style="font-size:30px" color="#ffffff"><b>PASSED</b></font></td>'

clear_output(wait=True)
html = html + conclusion + '</td></tr></table>'
get_ipython().run_cell_magic(u'HTML', u'', html)

admissions,23106,23106,
drugitems,4907269,4907269,
freetextitems,651248,651248,
listitems,30744065,30744065,
numericitems,977625612,977625612,
procedureorderitems,2188626,2188626,
processitems,256715,256715,
Verification:,,,PASSED


## Create Indices to increase performance
After verification this will run to create indices to improve query performance. However, this is a height process that can take hours depending on your system. In the meantime, the database is already to be queried (albeit slower), using the notebooks from the [tables](../tables/) folder.

In [13]:
import select
import psycopg2.extensions
from tqdm.notebook import tqdm
#from tqdm.notebook import tqdm
import time

def wait(conn):
    while True:
        state = conn.poll()
        if state == psycopg2.extensions.POLL_OK:
            break
        elif state == psycopg2.extensions.POLL_WRITE:
            select.select([], [conn.fileno()], [])
        elif state == psycopg2.extensions.POLL_READ:
            select.select([conn.fileno()], [], [])
        else:
            raise psycopg2.OperationalError("poll() returned %s" % state)

def get_status():
    status_sql = """
    SELECT c.relname as tablename, s.*, a.query FROM pg_stat_progress_create_index s 
    JOIN pg_stat_activity a ON s.pid = a.pid
    JOIN pg_class c on s.relid = c.oid
    WHERE query LIKE '%-- amsterdamumcdb indices%';
    """
    status = pd.read_sql(status_sql, con)
    if len(status) > 0:
        pid = status['pid'][0]
        phase = status['phase'][0]
        tablename = status['tablename'][0]
        
        if 'tuples' in phase:
            total = status['tuples_total'][0]
            current = status['tuples_done'][0]
        else:
            total = status['blocks_total'][0]
            current = status['blocks_done'][0]

        return pid, total, current, phase, tablename
    else:
        #no running index available
        return None, None, None, None, None
    

#create a new async connection, to monitor the status from another connection while the indexing is running.
aconn = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'], async_=1) 

wait(aconn) #wait until connection is ready
acurs = aconn.cursor()

index_sql = """
-- amsterdamumcdb indices
SET SCHEMA 'amsterdamumcdb';
-- admissions table
CREATE INDEX admissions_admissionid_index ON admissions (admissionid);
CREATE INDEX admissions_patientid_index ON admissions (patientid);

-- drugitems table
CREATE INDEX drugitems_admissionid_index ON drugitems (admissionid);
CREATE INDEX drugitems_orderid_index ON drugitems (orderid);
CREATE INDEX drugitems_ordercategoryid_index ON drugitems (ordercategoryid);
CREATE INDEX drugitems_itemid_index ON drugitems (itemid);
CREATE INDEX drugitems_start_index ON drugitems (start);
CREATE INDEX drugitems_stop_index ON drugitems (stop);

-- freetextitems table
CREATE INDEX freetextitems_admissionid_index ON freetextitems (admissionid);
CREATE INDEX freetextitems_itemid_index ON freetextitems (itemid);
CREATE INDEX freetextitems_measuredat_index ON freetextitems (measuredat);

-- listitems table
CREATE INDEX listitems_admissionid_index ON listitems (admissionid);
CREATE INDEX listitems_itemid_index ON listitems (itemid);
CREATE INDEX listitems_measuredat_index ON listitems (measuredat);

-- numericitems table
CREATE INDEX numericitems_admissionid_index ON numericitems (admissionid);
CREATE INDEX numericitems_itemid_index ON numericitems (itemid);
CREATE INDEX numericitems_measuredat_index ON numericitems (measuredat);
CREATE INDEX numericitems_admission_item_time_index ON numericitems (admissionid, itemid, measuredat);
CREATE INDEX numericitems_islabresult_index ON numericitems (islabresult);
CREATE INDEX numericitems_fluidout_index ON numericitems (fluidout);

-- procedureorderitems table
CREATE INDEX procedureorderitems_admissionid_index ON procedureorderitems (admissionid);
CREATE INDEX procedureorderitems_itemid_index ON procedureorderitems (itemid);
CREATE INDEX procedureorderitems_ordercategoryid_index ON procedureorderitems (ordercategoryid);
CREATE INDEX procedureorderitems_registeredat_index ON procedureorderitems (registeredat);

-- processitems table
CREATE INDEX processitems_admissionid_index ON processitems (admissionid);
CREATE INDEX processitems_itemid_index ON processitems (itemid);
CREATE INDEX processitems_start_index ON processitems (start);
CREATE INDEX processitems_stop_index ON processitems (stop);
"""
#execute create index sql query **asychronously**
acurs.execute(index_sql)

#wait 2 seconds to allow for starting up the indexing process 
time.sleep(2)

#progress bar variables
progress_blocks = None
progress_tuples = None
progress_current = None

#used for progress of indexing the database (based on indices in release 1.0.2)
TOTAL_INDEX_BLOCKS = 97361839
TOTAL_INDEX_TUPLES = 5.999210e+09

#get the current status of the indexing operation

pid, total, current, previous_phase, previous_tablename = get_status()
if pid == None:
    print('No indexing in progress.')
else:

    #create two tqdm objects for progress bars
    progress_blocks = tqdm(total=TOTAL_INDEX_BLOCKS, desc='Total blocks read', dynamic_ncols=True, 
                           unit_scale=1, unit='blocks') #total blocks progress
    progress_tuples = tqdm(total=TOTAL_INDEX_TUPLES, desc='Total tuples written', dynamic_ncols=True, 
                           unit_scale=1, unit='tuples') #total tuples progress
    progress_current = tqdm(total=total, desc='Processing table ' + previous_tablename + ' (' + previous_phase + ')',
                            dynamic_ncols=True, unit_scale=1, leave=False) #current index progress

    #update the progress bars every two seconds until the progress query returns empty or changes
    while True:  
            current_pid, total, current, phase, tablename = get_status()

            if pid == None or current_pid != pid:
                break #process changed: done

            #phase changed, if 'scanning table' a new item was started
            if (not previous_phase == phase) or \
                (not previous_tablename == tablename):
                
                reset = False
                if 'scanning table' in phase: #scanning tables, loading tuples in tree
                    #previous phase ('tuples') should be added completely to progress bar
                    progress_tuples.update(progress_current.total - progress_current.n)
                    reset = True
                    
                elif 'loading tuples in tree' in phase: #loading tuples in tree
                    #previous phase ('scanning table') should be added completely to progress bar
                    progress_blocks.update(progress_current.total - progress_current.n)          
                    reset = True
                
                if reset:
                    #finalize (100%) the progress bar, remove it and start a new one
                    progress_current.n = progress_current.total
                    progress_current.close()
                    progress_current = tqdm(total=total, desc='Processing table ' + tablename + ' (' + phase + ')',
                                            dynamic_ncols=True, unit_scale=1, leave=False) #current index progress
            
            #update progress bars based on the phase
            update = False
            if 'scanning table' in phase: #scanning tables, loading tuples in tree

                progress_current.unit = 'blocks'

                #update the progress of total blocks to read
                progress_blocks.update(current - progress_current.n)
                update = True

            elif 'loading tuples in tree' in phase:
                progress_current.unit = 'tuples'

                #update the progress of total tuples written
                progress_tuples.update(current - progress_current.n)
                update = True

            if update:
                #update the progress of the current index
                progress_current.n = current
                progress_current.update(0) #trigger refresh

                #store the current phase to compare with the next
                previous_phase = phase
                previous_tablename = tablename

            
            time.sleep(1) #wait for one second before requerying indexing progress

    #left while loop
    if not progress_blocks is None:
        progress_blocks.n = TOTAL_INDEX_BLOCKS
        progress_blocks.close()
    if not progress_tuples is None:
        progress_tuples.n = TOTAL_INDEX_TUPLES
        progress_tuples.close()
    if not progress_current is None:
        progress_current.n = progress_current.total
        progress_current.close()
        
    print('Indexing done.')
    
wait(aconn) #wait until connection is ready
aconn.close()

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Total blocks read:   0%|          | 0.00/97.4M [00:00<?, ?blocks/s]

Total tuples written:   0%|          | 0.00/6.00G [00:00<?, ?tuples/s]

Processing table drugitems (building index: scanning table):   0%|          | 0.00/152k [00:00<?, ?it/s]

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


...

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas 

Processing table listitems (building index: loading tuples in tree):   0%|          | 0.00/30.7M [00:00<?, ?it…

/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/akigroup/anaconda3/envs/data/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Indexing done.


KeyboardInterrupt: 